In [1]:
%iam_role arn:aws:iam::935670829844:role/dwhR
%glue_version 3.0
%region us-east-1
print("Ready to develop")

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.4 and you have 1.0.1 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::935670829844:role/dwhR.
Setting Glue version to: 3.0
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 5
Session ID: 1b3c82ee-ca2c-4b76-ab62-b0e520af0a1d
Applying the following default arguments:
--glue_kernel_version 1.0.1
--enable-glue-datacatalog true
Waiting for session 1b3c8

In [ ]:
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.sql.functions import col, expr
from awsglue.dynamicframe import DynamicFrame
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import sys

args = getResolvedOptions(sys.argv, ["JOB_NAME"])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args["JOB_NAME"], args)

# Script generated for node Amazon S3
AmazonS3_node1702362906597 = glueContext.create_dynamic_frame.from_catalog(
    database="people_db1",
    table_name="input_large_csv",
    transformation_ctx="AmazonS3_node1702362906597",
)

# Print the schema of the DynamicFrame
print("Schema of AmazonS3_node1702362906597:")
print(AmazonS3_node1702362906597.printSchema())

# Transformation 1: Filter records where age is greater than 25
transformed_data_1 = Filter.apply(frame=AmazonS3_node1702362906597, f=lambda x: x["age"] > 25)

# Transformation: Add a new column 'city_category' based on city
transformed_data_1 = transformed_data_1.toDF()
transformed_data = transformed_data_1.withColumn(
    "city_category",
    expr("CASE WHEN city IN ('London', 'Paris') THEN 'Europe' ELSE 'Other' END")
)

# Convert back to DynamicFrame
transformed_dynamic_frame = DynamicFrame.fromDF(transformed_data, glueContext, "transformed_data")


# Write the final transformed data to an S3 location in CSV format
glueContext.write_dynamic_frame.from_options(
    frame=transformed_dynamic_frame,
    connection_type="s3",
    connection_options={"path": "s3://ieshaan-bucket-latest/output_data/"},
    format="csv",
    format_options={"compression": "SNAPPY", "writeHeader": True}  # Ensure writeHeader is set to True
)

job.commit()
